#Criar um script que faça:  
#Pegue as nfe em xml do email  
#Extrai os dados necessarios   
#Tratar dados   
#Aplicador formula de taxas  
#Inserir no template dos marketplaces
  

In [13]:
import os
import shutil
import xml.etree.ElementTree as ET
import pandas as pd
from imap_tools import MailBox, AND
from config import pwd, user
from datetime import date
import openpyxl

In [ ]:
#Baixa os anexos XML dos emails
save_folder = "notas/nfes/"
os.makedirs(save_folder, exist_ok=True)

with MailBox("imap.gmail.com").login(user, pwd, initial_folder="INBOX", ) as mailbox:
    list_mail = mailbox.fetch(criteria=AND(date=date.today())

    )
    for email in list_mail:
        for anexo in email.attachments:
            if  anexo.filename.lower().endswith(".xml"):
                file_path = os.path.join(save_folder, anexo.filename)
                if not os.path.exists(file_path):   
                    with open(file_path, "wb") as f:
                        f.write(anexo.payload)

In [ ]:
#Salva os XMLs em pastas por Emitente e extrai os produtos para um Excel

pasta_origem = "notas/nfes"
todos_produtos = []


def extrai_dados(caminho_arquivo):
    produtos = []
    tree = ET.parse(caminho_arquivo)
    root = tree.getroot()
    ns = {"ns": "http://www.portalfiscal.inf.br/nfe"}
    
    natOp = root.find(".//ns:ide/ns:natOp", ns)
    if natOp is not None and natOp.text.strip().upper() == 'BONIFICACAO, DOACAO OU BRINDE':
        return [] 
    
    for det in root.findall(".//ns:det", ns):   
        produto = {}
        produto ['Descrição'] = det.find("./ns:prod/ns:xProd", ns).text
        produto ['Valor_unitário'] = det.find("./ns:prod/ns:vUnCom", ns).text
        produto ['Código de Barras'] = det.find("./ns:prod/ns:cEAN", ns).text 
        produto ['Sku'] = produto['Código de Barras']
        if produto['Sku'] == 'SEM GTIN':
            produto['Sku'] = produto['Descrição']
        produto ['Fornecedor'] = root.find(".//ns:emit/ns:xNome", ns).text
        produtos.append(produto)
    return produtos
        


# Percorre todos os arquivos da pasta
for arquivo in os.listdir(pasta_origem):
    if arquivo.lower().endswith(".xml"):
        caminho_arquivo = os.path.join(pasta_origem, arquivo)

        try:
            # Abre e parseia o XML
            tree = ET.parse(caminho_arquivo)
            root = tree.getroot()

            # Define namespace
            ns = {"ns": "http://www.portalfiscal.inf.br/nfe"}

            # Busca o CNPJ do emitente
            cnpj_emitente = root.find(".//ns:emit/ns:CNPJ", ns)
            cnpj_emitente
            if cnpj_emitente is None:
                print(f"⚠️ CNPJ não encontrado em {arquivo}")
                continue

            cnpj_emitente = cnpj_emitente.text

            produtos = extrai_dados(caminho_arquivo)
            todos_produtos.extend(produtos)              


            emitente = root.find(".//ns:emit/ns:xNome", ns)
            pasta_destino = os.path.join("notas/", emitente.text)

            # Cria a pasta se não existir
            os.makedirs(pasta_destino, exist_ok=True)

            # Copia o arquivo para a pasta
            shutil.copy(caminho_arquivo, pasta_destino)

        except Exception as e:
            print(f" {arquivo}: {e}")


df = pd.DataFrame(todos_produtos)


final = df.drop_duplicates(subset=['Sku'], keep='first')

final


final.to_excel("produtos_extraidos.xlsx", index=False)
sku = df.query('Sku == Descrição')
sku.to_excel("skus.xlsx", index=False)

In [31]:
final

,Descrição,Valor_unitário,Código de Barras,Sku,Fornecedor
0,"BROCA ACO RAPIDO DEWALT 2,0MM",2.451,7896525070801,7896525070801,MJR CUNHA DISTRIBUIDORA DE MAT.CONSTR LTDA
1,"BROCA ACO RAPIDO DEWALT 2,5MM",1.958,7896525069133,7896525069133,MJR CUNHA DISTRIBUIDORA DE MAT.CONSTR LTDA
2,SUPORTE BRASFORMA P/FORNO BR SBR3.6,23.206,7896689110368,7896689110368,MJR CUNHA DISTRIBUIDORA DE MAT.CONSTR LTDA
3,ASSENTO METASUL ALMOFADADO OVAL BRANCO,29.69,7898188095115,7898188095115,MJR CUNHA DISTRIBUIDORA DE MAT.CONSTR LTDA
4,COTOVELO ESGOTO FORTLEV B 50 X 90 BR,1.05,7898543592662,7898543592662,MJR CUNHA DISTRIBUIDORA DE MAT.CONSTR LTDA
...,...,...,...,...,...
9585,PREGO ACO 10X10 C/C KG**,77.9930000000,SEM GTIN,PREGO ACO 10X10 C/C KG**,CONSTRUJA DISTR. DE MATERIAIS P/ CONSTRU
9586,"MASSA RAPIDA CZ 1,25KG",37.6650000000,7898031540618,7898031540618,CONSTRUJA DISTR. DE MATERIAIS P/ CONSTRU
9588,PREGO ACO 18X24 C/C %,32.6690000000,7898562870192,7898562870192,CONSTRUJA DISTR. DE MATERIAIS P/ CONSTRU
9590,SUP EXT ZINCO C/33 PLATIBANDA,9.1940000000,7897871104127,7897871104127,CONSTRUJA DISTR. DE MATERIAIS P/ CONSTRU
